# Пробуем разнообразить способ с Вебинара №2

In [1]:
!pip install torch
!pip install transformers datasets

     ------------------------------------ 167.3/167.3 MB 829.3 kB/s eta 0:00:00

[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
     ---------------------------------------- 5.8/5.8 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 451.7/451.7 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 182.4/182.4 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 324.3/324.3 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 78.5/78.5 kB 874.7 kB/s eta 0:00:00
     ------------------------------------ 139.5/139.5 kB 916.5 kB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 20.3/20.3 MB 979.9 kB/s eta 0:00:00
     -------------------------------------- 56.9/56.9 kB 751.9 kB/s eta 0:00:00
  

In [2]:
import json
from pprint import pprint
import re

import pandas as pd
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from transformers import AutoModel, AutoTokenizer

In [12]:
import numpy as np

In [3]:
class BertEmbedder(BaseEstimator, TransformerMixin):
    """Получаете эмбеддинги для батча текстов"""

    def __init__(self, bert_tokenizer, bert_model):
        self.bert_tokenizer = bert_tokenizer
        self.bert_model = bert_model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        with torch.no_grad():
            t = self.bert_tokenizer(
                X.tolist(), padding=True, truncation=True, return_tensors="pt"
            )

            model_output = self.bert_model(**t)
            embeddings = model_output.last_hidden_state[:, 0, :]
            embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.numpy()

In [27]:
train = pd.read_csv("../Data/train_fd.csv", index_col="index")
train = train.dropna()
train = train[:3000]
train
x_train = train.drop(columns=["target"])
y_train = train["target"]

In [22]:
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [23]:
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
%%time
clf_bert = Pipeline(
    [
        
        (
            "bert",
            # изменен способ векторизации текстов
            ColumnTransformer([("vectorize", BertEmbedder(rubert_tokenizer, rubert_model), "full_description")]),
        ),
        (
            "clf",
            # заменена модель-классификатор
            MLPClassifier(
                hidden_layer_sizes=(64, 64), random_state=20221202, max_iter=500
            ),
        ),
    ]
)

clf_bert.fit(x_train, y_train)
print(clf_bert.score(x_train, y_train))

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 55890576000 bytes.